# Optimal portfolios

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import minimize
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions

### Inputs

In [2]:
##### Inputs
# Risk-free rate
RF = 0.02

# Expected returns
MNS = np.array([0.06, 0.035])

# Standard deviations
SDS = np.array([0.15, 0.035])

# Correlations
C  = np.identity(2)
C[0, 1] = C[1, 0] = -0.05

# Covariance matrix
COV = np.diag(SDS) @ C @ np.diag(SDS)
print(COV)

[[ 0.0225    -0.0002625]
 [-0.0002625  0.001225 ]]


### Tangency portfolio

In [3]:
n = len(MNS)
def f(w):
    mn = w @ MNS
    sd = np.sqrt(w @ COV @ w)
    return -(mn - RF) / sd
# Initial guess (equal-weighted)
w0 = (1/n)*np.ones(n)
# Constraint: fully-invested portfolio
A = np.ones(n)
b = 1
cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
# No short-sale constraint
bnds = [(None, None) for i in range(n)] 
# Optimization
TOL = 10**(-10)
wgts = minimize(f, w0, bounds=bnds, constraints=cons, options={'ftol':TOL}).x

print(f'Tangency portfolio weights: {wgts}')

Tangency portfolio weights: [0.13203413 0.86796587]


### Tangency portfolio: theoretical solution without short-sale constraint

In [4]:
w = np.linalg.solve(COV, MNS - RF)
wgts = w / np.sum(w)
print(f'Tangency portfolio weights: {wgts}')

Tangency portfolio weights: [0.13203429 0.86796571]


In [5]:
#Portfolio expected return
port_expret = wgts @ MNS
print(f'Portfolio Expected Return:\t {port_expret: ,.4f}')

# Portfolio standard deviation
port_sd = np.sqrt(wgts @ COV @ wgts)
print(f'Portfolio Standard Deviation:\t {port_sd: ,.4f}')

# Portfolio sharpe ratio
port_sr = (port_expret - RF)/port_sd
print(f'Portfolio Sharpe Ratio:\t\t {port_sr: ,.4f}')

Portfolio Expected Return:	  0.0383
Portfolio Standard Deviation:	  0.0354
Portfolio Sharpe Ratio:		  0.5166


### Capital allocation and risk aversion

In [6]:
def capital_allocation(mean, sd, rf, risk_aversion):
    return (mean - rf) / (risk_aversion * (sd**2))

In [7]:
RAVER = 10
wgt_risky = capital_allocation(port_expret, port_sd, RF, RAVER)
print(f'For risk aversion of {RAVER: .1f}, invest {wgt_risky: ,.1%} in the risky asset')

For risk aversion of  10.0, invest  145.8% in the risky asset


In [8]:
# Portfolio expected return
optport_expret = wgt_risky*port_expret + (1-wgt_risky)*RF
print(f'For risk aversion of {RAVER: .1f}, Portfolio Expected Return:\t\t {optport_expret: ,.4f}')

# Portfolio standard deviation
optport_sd     = np.abs(wgt_risky)*port_sd
print(f'For risk aversion of {RAVER: .1f}, Portfolio Standard Deviation:\t {optport_sd: ,.4f}')

For risk aversion of  10.0, Portfolio Expected Return:		  0.0467
For risk aversion of  10.0, Portfolio Standard Deviation:	  0.0517


### Capital allocation line

In [9]:
WGTS = np.arange(0,2.51,0.05)
cal = pd.DataFrame(dtype='float',columns=['wgt_tangency', 'wgt_rf', 'port_expret', 'port_sd','risk_aversion'], index=np.arange(len(WGTS)))
cal.wgt_tangency = WGTS
cal.wgt_rf = 1-WGTS
cal.port_expret = cal.wgt_tangency*port_expret + cal.wgt_rf*RF
cal.port_sd = np.abs(cal.wgt_tangency)*port_sd
cal.risk_aversion = (port_expret - RF) / (cal.wgt_tangency * port_sd**2)
cal.tail()

,wgt_tangency,wgt_rf,port_expret,port_sd,risk_aversion
46,2.30,-1.30,0.062092,0.081478,6.340410
47,2.35,-1.35,0.063007,0.083249,6.205507
48,2.40,-1.40,0.063922,0.085021,6.076226
49,2.45,-1.45,0.064837,0.086792,5.952221
50,2.50,-1.50,0.065752,0.088563,5.833177


### The full picture
Note we will now also calculate the sharpe ratio for each frontier portfolio

In [10]:
# Calculate frontier portfolios (from last time)
def frontier(means, cov, target):
    n = len(means)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    # Constraint: short-sales allowed
    G = matrix(np.zeros((n,n)), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    # Fully-invested constraint + E[r] = target
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, target], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    wgts = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts
SolverOptions['show_progress'] = False

NUM_TARGETS = 30
df = pd.DataFrame(dtype='float',columns=['target_expret','w1','w2','port_expret','port_sd','sharpe'], index=np.arange(NUM_TARGETS))
df.target_expret = np.linspace(MNS.min(), MNS.max(),NUM_TARGETS)
for i in df.index:
    wgts = frontier(MNS, COV, df.loc[i,'target_expret'])
    df.loc[i,['w1','w2']] = wgts
    df.loc[i,'port_expret'] = wgts @ MNS
    df.loc[i,'port_sd'] = np.sqrt(wgts @ COV @ wgts)
df.sharpe = (df.port_expret - RF)/df.port_sd

In [11]:
fig = go.Figure()

# Plot frontier
string =  "Stock: %{customdata[0]:.1%}<br>"
string += "Bond: %{customdata[1]:.1%}<br>"
string += "Sharpe ratio: %{customdata[2]:.4f}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=df.port_sd, y=df.port_expret,mode="lines",
    customdata=df[['w1','w2','sharpe']],hovertemplate=string, name='Frontier')
fig.add_trace(trace)

# Plot underlying assets
trace1= go.Scatter(x=[SDS[0]], y=[MNS[0]],mode="markers", marker=dict(size=10, color="red"), name='Stock')
fig.add_trace(trace1)
trace2= go.Scatter(x=[SDS[1]], y=[MNS[1]],mode="markers", marker=dict(size=10, color="red"), name='Bond')
fig.add_trace(trace2)

# Plot tangency
trace= go.Scatter(x=[port_sd], y=[port_expret] ,
    mode="markers", marker=dict(size=10, color="black"),
    customdata=np.append(wgts,port_sr).reshape(1,3),hovertemplate=string, name='Tangency')
fig.add_trace(trace)

# Plot CAL
string =  "Tangency: %{customdata[0]:.1%}<br>"
string += "Risk-free: %{customdata[1]:.1%}<br>"
string += "Optimal if risk aversion is: %{customdata[2]:.1f}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=cal.port_sd, y=cal.port_expret,mode="lines",marker=dict(color="black"),
    customdata=cal[['wgt_tangency','wgt_rf','risk_aversion']],hovertemplate=string, name='CAL')
fig.add_trace(trace)

# Plot optimal location on CAL for risk aversion
string =  "Tangency: %{customdata[0]:.1%}<br>"
string += "Risk-free: %{customdata[1]:.1%}<br>"
string += "Optimal if risk aversion is: %{customdata[2]:.1f}<br>"
string += "<extra></extra>"
trace= go.Scatter(x=[optport_sd], y=[optport_expret],
    mode="markers", marker=dict(size=15, color="red", symbol='star'),
    customdata=[[wgt_risky, 1-wgt_risky, RAVER]],hovertemplate=string, name='Optimal Portfolio')
fig.add_trace(trace)

# Formatting
fig.layout.yaxis["title"] = "Expected Return"
fig.layout.xaxis["title"] = "Standard Deviation"
fig.update_yaxes(tickformat=".1%")
fig.update_xaxes(tickformat=".1%")
fig.update_xaxes(range=[0.7 * df["port_sd"].min(), 1.25 * df["port_sd"].max()])
fig.update_yaxes(range=[0.7 * df["port_expret"].min(), 1.25 * df["port_expret"].max()])
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()